# Training a model for catchment classification. 

# Imports

In [1]:
import swmmio
import pyswmm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

desired_width = 500
pd.set_option("display.width", desired_width)
np.set_printoptions(linewidth=desired_width)
pd.set_option("display.max_columns", 30)

In [2]:
classes = pd.DataFrame(
    data={
        "classes": [
            "compact_urban_development",
            "urban",
            "loose_urban_development",
            "wooded_area",
            "grassy",
            "loose_soil",
            "steep_area",
        ]
    }
)
classes

,classes
0,compact_urban_development
1,urban
2,loose_urban_development
3,wooded_area
4,grassy
5,loose_soil
6,steep_area


# Get files

In [3]:
INP_FILE = "dataset/subcatchment_dataset.inp"
RPT_FILE = "dataset/subcatchment_dataset.inp"

# Run simulation

In [6]:
with pyswmm.Simulation(INP_FILE) as sim:
    for step in sim:
        pass

# Read inp and rpt file as swmmio model object

In [30]:
model = swmmio.Model(INP_FILE)

## Get subcatchments data from the model

In [31]:
raw_subcatchments = model.subcatchments.dataframe
subcatchments = raw_subcatchments.copy()
subcatchments[:5]

,Raingage,Outlet,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,RouteTo,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff,coords
Name,,,,,,,,,,,,,,,,,,,,,,,,
S1,Raingage2,O1,0.50,20.00,300.00,5.00,0,0.150,0.41,1.27,5.08,70,OUTLET,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407,"[(777180.0, 592590.0), (777180.0, 592585.0), (777175.0, 592585.0), (777175.0, 592590.0), (777180.0, 592590.0)]"
S10,Raingage2,O1,1.87,45.00,136.75,15.00,0,0.013,0.15,1.27,5.08,90,OUTLET,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587,NaN
S100,Raingage2,O1,1.71,45.00,130.77,10.00,0,0.013,0.15,1.27,5.08,90,OUTLET,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581,NaN
S1000,Raingage2,O1,0.68,10.00,82.46,5.09,0,0.150,0.41,1.27,5.08,10,OUTLET,10.10,0.00,0.00,7.73,0.90,1.36,2.26,0.02,0.01,0.224,NaN
S1001,Raingage2,O1,0.26,83.33,50.99,45.00,0,0.013,0.05,1.27,5.08,80,OUTLET,10.10,0.00,0.00,1.04,8.23,0.65,8.88,0.02,0.02,0.879,NaN


### Drop unused columns

In [32]:
subcatchments.drop(['coords', 'RouteTo'], axis=1, inplace=True)
subcatchments

,Raingage,Outlet,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,,,,,,,,,,,,
S1,Raingage2,O1,0.50,20.00,300.00,5.00,0,0.150,0.41,1.27,5.08,70,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407
S10,Raingage2,O1,1.87,45.00,136.75,15.00,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587
S100,Raingage2,O1,1.71,45.00,130.77,10.00,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581
S1000,Raingage2,O1,0.68,10.00,82.46,5.09,0,0.150,0.41,1.27,5.08,10,10.10,0.00,0.00,7.73,0.90,1.36,2.26,0.02,0.01,0.224
S1001,Raingage2,O1,0.26,83.33,50.99,45.00,0,0.013,0.05,1.27,5.08,80,10.10,0.00,0.00,1.04,8.23,0.65,8.88,0.02,0.02,0.879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S995,Raingage2,O1,1.10,10.00,104.88,15.00,0,0.150,0.41,1.27,5.08,10,10.10,0.00,0.00,7.53,0.90,1.56,2.46,0.03,0.02,0.243
S996,Raingage2,O1,1.89,5.09,137.48,15.00,0,0.060,0.17,1.27,5.08,10,10.10,0.00,0.00,7.55,0.46,2.04,2.49,0.05,0.03,0.247
S997,Raingage2,O1,1.19,10.00,109.09,5.09,0,0.150,0.41,1.27,5.08,10,10.10,0.00,0.00,7.91,0.90,1.18,2.08,0.02,0.01,0.206


## Get categories

In [33]:
categories = model.inp.tags
categories

,Name,Tag
ElementType,,
Subcatch,S1,#comment
Subcatch,S2,compact_urban_development
Subcatch,S3,loose_soil
Subcatch,S4,wooded_area
Subcatch,S5,loose_urban_development
...,...,...
Subcatch,S997,grassy
Subcatch,S998,steep_area
Subcatch,S999,urban


### Add categories column to subcatchments DataFrame

In [35]:
# Reset the index of subcatchments DataFrame
subcatchments.reset_index(inplace=True)

# Merge the two DataFrames on the "Name" column
merged_df = subcatchments.merge(model.inp.tags, left_on="Name", right_on="Name", how="left")

# Rename the 'Tag' column to 'categories'
merged_df.rename(columns={"Tag": "categories"}, inplace=True)

# Set the index back to "Name"
merged_df.set_index("Name", inplace=True)

In [36]:
merged_df[:3]

,index,Raingage,Outlet,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff,categories
Name,,,,,,,,,,,,,,,,,,,,,,,,
S1,0,Raingage2,O1,0.50,20.0,300.00,5.0,0,0.150,0.41,1.27,5.08,70,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407,#comment
S10,1,Raingage2,O1,1.87,45.0,136.75,15.0,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587,compact_urban_development
S100,2,Raingage2,O1,1.71,45.0,130.77,10.0,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581,compact_urban_development


In [37]:
merged_df.head()

,index,Raingage,Outlet,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff,categories
Name,,,,,,,,,,,,,,,,,,,,,,,,
S1,0,Raingage2,O1,0.50,20.00,300.00,5.00,0,0.150,0.41,1.27,5.08,70,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407,#comment
S10,1,Raingage2,O1,1.87,45.00,136.75,15.00,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587,compact_urban_development
S100,2,Raingage2,O1,1.71,45.00,130.77,10.00,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581,compact_urban_development
S1000,3,Raingage2,O1,0.68,10.00,82.46,5.09,0,0.150,0.41,1.27,5.08,10,10.10,0.00,0.00,7.73,0.90,1.36,2.26,0.02,0.01,0.224,grassy
S1001,4,Raingage2,O1,0.26,83.33,50.99,45.00,0,0.013,0.05,1.27,5.08,80,10.10,0.00,0.00,1.04,8.23,0.65,8.88,0.02,0.02,0.879,steep_area


### Split data into features and target

In [38]:
X = merged_df.drop('categories', axis=1)
y = merged_df['categories']
print(X.shape, y.shape)

(1001, 23) (1001,)


In [39]:
X["TotalPrecip"] = pd.to_numeric(X["TotalPrecip"])
X["TotalRunon"] = pd.to_numeric(X["TotalRunon"])
X["TotalEvap"] = pd.to_numeric(X["TotalEvap"])
X["TotalInfil"] = pd.to_numeric(X["TotalInfil"])
X["PervRunoff"] = pd.to_numeric(X["PervRunoff"])

In [40]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1001 entries, S1 to S999
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          1001 non-null   int64  
 1   Raingage       1001 non-null   object 
 2   Outlet         1001 non-null   object 
 3   Area           1001 non-null   float64
 4   PercImperv     1001 non-null   float64
 5   Width          1001 non-null   float64
 6   PercSlope      1001 non-null   float64
 7   CurbLength     1001 non-null   int64  
 8   N-Imperv       1001 non-null   float64
 9   N-Perv         1001 non-null   float64
 10  S-Imperv       1001 non-null   float64
 11  S-Perv         1001 non-null   float64
 12  PctZero        1001 non-null   int64  
 13  TotalPrecip    1001 non-null   float64
 14  TotalRunon     1001 non-null   float64
 15  TotalEvap      1001 non-null   float64
 16  TotalInfil     1001 non-null   float64
 17  ImpervRunoff   1001 non-null   float64
 18  PervRunoff  

In [41]:
X = pd.get_dummies(X, drop_first=True)
X.head()

,index,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,,,,,,,,,,,
S1,0,0.50,20.00,300.00,5.00,0,0.150,0.41,1.27,5.08,70,10.1,0.0,0.0,5.91,1.95,2.17,4.12,0.02,0.02,0.407
S10,1,1.87,45.00,136.75,15.00,0,0.013,0.15,1.27,5.08,90,10.1,0.0,0.0,4.13,4.50,1.43,5.93,0.11,0.09,0.587
S100,2,1.71,45.00,130.77,10.00,0,0.013,0.15,1.27,5.08,90,10.1,0.0,0.0,4.18,4.50,1.38,5.87,0.10,0.08,0.581
S1000,3,0.68,10.00,82.46,5.09,0,0.150,0.41,1.27,5.08,10,10.1,0.0,0.0,7.73,0.90,1.36,2.26,0.02,0.01,0.224
S1001,4,0.26,83.33,50.99,45.00,0,0.013,0.05,1.27,5.08,80,10.1,0.0,0.0,1.04,8.23,0.65,8.88,0.02,0.02,0.879


In [42]:
# Create a MinMaxScaler instance
scaler = MinMaxScaler()

# Fit the scaler to the data and transform the data
X_normalized = scaler.fit_transform(X)

# Convert the normalized data back to a dataframe with the same column names
X_normalized_df = pd.DataFrame(X_normalized, columns=X.columns, index=X.index)

In [43]:
X_normalized_df.head()

,index,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,,,,,,,,,,,
S1,0.000,0.246231,0.218148,1.000000,0.044995,0.0,0.354005,0.480000,0.0,0.0,0.764706,0.0,0.0,0.0,0.608479,0.216687,0.683871,0.392630,0.111111,0.142857,0.391528
S10,0.001,0.934673,0.526790,0.437069,0.213515,0.0,0.000000,0.133333,0.0,0.0,1.000000,0.0,0.0,0.0,0.386534,0.534247,0.445161,0.622618,0.611111,0.642857,0.622593
S100,0.002,0.854271,0.526790,0.416448,0.129255,0.0,0.000000,0.133333,0.0,0.0,1.000000,0.0,0.0,0.0,0.392768,0.534247,0.429032,0.614994,0.555556,0.571429,0.614891
S1000,0.003,0.336683,0.094691,0.249862,0.046512,0.0,0.354005,0.480000,0.0,0.0,0.058824,0.0,0.0,0.0,0.835411,0.085928,0.422581,0.156290,0.111111,0.071429,0.156611
S1001,0.004,0.125628,1.000000,0.141345,0.719077,0.0,0.000000,0.000000,0.0,0.0,0.882353,0.0,0.0,0.0,0.001247,0.998755,0.193548,0.997459,0.111111,0.142857,0.997433


### Encode target labels

In [44]:
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))

In [45]:
y_encoded

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

# Split the data into training and testing sets

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized_df, y_encoded, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (800, 21)
y_train shape: (800, 8)
X_test shape: (201, 21)
y_test shape: (201, 8)


# Build the model

In [47]:
model = Sequential()
model.add(Dense(16, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1], activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 16)                352       
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_4 (Dense)             (None, 8)                 136       
                                                                 
 dropout_3 (Dropout)         (None, 8)                 0         
                                                                 
 dense_5 (Dense)             (None, 8)                 72        
                                                                 
Total params: 560
Trainable params: 560
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=100,
    validation_data=(X_test, y_test),
    validation_split=0.2,
    verbose=1,
    batch_size=16,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10)]
)

Epoch 1/100
50/50 [==============================] - 1s 7ms/step - loss: 0.2069 - accuracy: 0.9262 - val_loss: 0.0318 - val_accuracy: 1.0000
Epoch 2/100
50/50 [==============================] - 0s 3ms/step - loss: 0.1892 - accuracy: 0.9438 - val_loss: 0.0312 - val_accuracy: 1.0000
Epoch 3/100
50/50 [==============================] - 0s 3ms/step - loss: 0.1656 - accuracy: 0.9575 - val_loss: 0.0287 - val_accuracy: 1.0000
Epoch 4/100
50/50 [==============================] - 0s 3ms/step - loss: 0.1747 - accuracy: 0.9463 - val_loss: 0.0297 - val_accuracy: 1.0000
Epoch 5/100
50/50 [==============================] - 0s 3ms/step - loss: 0.2116 - accuracy: 0.9262 - val_loss: 0.0299 - val_accuracy: 1.0000
Epoch 6/100
50/50 [==============================] - 0s 3ms/step - loss: 0.1717 - accuracy: 0.9413 - val_loss: 0.0277 - val_accuracy: 1.0000
Epoch 7/100
50/50 [==============================] - 0s 3ms/step - loss: 0.2060 - accuracy: 0.9337 - val_loss: 0.0267 - val_accuracy: 1.0000
Epoch 8/100
5

In [50]:
# Evaluate ANN model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test set accuracy: {accuracy:.2f}")

7/7 [==============================] - 0s 2ms/step - loss: 0.0221 - accuracy: 1.0000
Test set accuracy: 1.00


In [51]:
# model.predict(X_test)
y_pred = model.predict(X_test)

7/7 [==============================] - 0s 1ms/step


In [55]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics

,loss,accuracy,val_loss,val_accuracy,epoch
0,0.206904,0.92625,0.031814,1.0,0
1,0.189153,0.94375,0.031194,1.0,1
2,0.165623,0.95750,0.028698,1.0,2
3,0.174714,0.94625,0.029673,1.0,3
4,0.211622,0.92625,0.029912,1.0,4
5,0.171678,0.94125,0.027667,1.0,5
6,0.205983,0.93375,0.026674,1.0,6
7,0.202368,0.93250,0.029005,1.0,7
8,0.206868,0.94375,0.030106,1.0,8
9,0.206749,0.93375,0.030698,1.0,9


# Model evaluate

In [56]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['accuracy'], name='accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['loss'], name='loss'), row=1, col=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_accuracy'], name='val_accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_loss'], name='val_loss'), row=1, col=2)

fig.update_xaxes(title_text='epochs')
fig.update_yaxes(title_text='accuracy')
fig.update_layout(width=1000, title='Accuracy and Loss')
fig.show()

In [57]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(test_acc)

1.0


Predykcja na podstawie modelu.
* model.evaluate(y_true, y_pred) - pozwala obliczyć metryki modelu
* model.predict_classes() - pozwala zwrócić odpowiednio przewidziane klasy
* model.predict_proba(), model.predict() - pozwala zwrócić prawdopodobieństwo danej klasy

In [58]:
predictions = model.predict(X_test)
predictions

7/7 [==============================] - 0s 2ms/step


array([[3.3899686e-10, 1.9547931e-27, 9.9999857e-01, ..., 2.0271515e-18, 9.5899244e-17, 2.6894371e-09],
       [6.4157421e-04, 6.9083718e-08, 2.7667466e-03, ..., 2.0614530e-06, 4.0034614e-05, 2.9122733e-04],
       [1.5092069e-09, 6.2632665e-26, 9.9999821e-01, ..., 1.2528053e-16, 5.1928947e-15, 2.4766043e-09],
       ...,
       [2.3087073e-22, 8.9350849e-30, 1.5581450e-06, ..., 4.6803438e-30, 2.6182833e-38, 9.9999845e-01],
       [1.2802441e-08, 4.8800103e-23, 9.9999595e-01, ..., 1.9499300e-14, 4.1101389e-13, 1.3439305e-08],
       [1.9302787e-03, 1.1111683e-06, 9.5764957e-03, ..., 3.9356881e-05, 8.9573540e-04, 6.0943735e-04]], dtype=float32)

In [60]:
predictions_df = pd.DataFrame(predictions)
predictions_df

,0,1,2,3,4,5,6,7
0,3.389969e-10,1.954793e-27,0.999999,1.958734e-09,1.409109e-06,2.027151e-18,9.589924e-17,2.689437e-09
1,6.415742e-04,6.908372e-08,0.002767,4.848462e-04,9.957734e-01,2.061453e-06,4.003461e-05,2.912273e-04
2,1.509207e-09,6.263267e-26,0.999998,5.867821e-08,1.618125e-06,1.252805e-16,5.192895e-15,2.476604e-09
3,7.703693e-10,9.869490e-27,0.999999,3.117990e-08,1.248597e-06,4.160892e-17,1.564857e-15,1.227637e-09
4,1.398058e-08,9.128074e-23,0.999990,1.088876e-07,9.614505e-06,3.278491e-15,1.097086e-13,5.953532e-08
...,...,...,...,...,...,...,...,...
196,3.033864e-21,6.070379e-29,0.000004,1.729010e-32,3.022138e-22,2.185718e-29,1.134334e-36,9.999957e-01
197,1.457512e-08,4.803436e-23,0.999996,1.340204e-06,3.002123e-06,2.810331e-14,6.904697e-13,1.086806e-08
198,2.308707e-22,8.935085e-30,0.000002,6.241395e-34,1.388100e-23,4.680344e-30,2.618283e-38,9.999985e-01
199,1.280244e-08,4.880010e-23,0.999996,7.290522e-07,3.363181e-06,1.949930e-14,4.110139e-13,1.343930e-08


In [62]:
predictions_cls = predictions.argmax(axis=-1)
predictions_cls

array([2, 4, 2, 2, 2, 1, 2, 7, 3, 2, 3, 7, 2, 5, 5, 1, 5, 4, 2, 3, 7, 3, 7, 4, 7, 1, 2, 4, 6, 1, 2, 6, 6, 4, 6, 1, 7, 2, 7, 4, 2, 5, 7, 2, 2, 2, 7, 2, 3, 4, 1, 1, 4, 1, 3, 2, 7, 2, 7, 3, 2, 7, 2, 6, 7, 2, 3, 1, 3, 2, 2, 2, 4, 2, 2, 2, 1, 2, 2, 1, 7, 2, 2, 2, 6, 2, 5, 7, 2, 7, 6, 2, 3, 4, 2, 2, 2, 2, 6, 6, 3, 2, 6, 4, 5, 3, 6, 3, 2, 2, 1, 2, 2, 2, 6, 7, 3, 1, 2, 2, 6, 7, 2, 2, 6, 3, 2, 2, 3, 5, 4, 1, 6, 2, 1, 4, 7, 3, 3, 7, 6, 2, 7, 2, 2, 2, 4, 2, 4, 2, 3, 5, 2, 6, 1, 2, 5, 2, 4, 2, 4, 6, 2, 3,
       6, 5, 2, 1, 4, 2, 6, 2, 1, 5, 1, 2, 4, 2, 3, 2, 7, 1, 2, 3, 5, 6, 6, 1, 4, 2, 2, 2, 7, 7, 2, 1, 7, 2, 7, 2, 4], dtype=int64)

# Zapisanie modelu

In [64]:
model.save(r'C:\Users\Dell\Documents\Git\stormwater-analysis\data')

INFO:tensorflow:Assets written to: C:\Users\Dell\Documents\Git\stormwater-analysis\data\assets


INFO:tensorflow:Assets written to: C:\Users\Dell\Documents\Git\stormwater-analysis\data\assets


# Załadowanie modelu

In [68]:
from tensorflow import keras
model = keras.models.load_model(r'C:\Users\Dell\Documents\Git\stormwater-analysis\data')

In [69]:
pred = model.predict(X_test)
pred

7/7 [==============================] - 0s 2ms/step


array([[3.3899686e-10, 1.9547931e-27, 9.9999857e-01, ..., 2.0271515e-18, 9.5899244e-17, 2.6894371e-09],
       [6.4157421e-04, 6.9083718e-08, 2.7667466e-03, ..., 2.0614530e-06, 4.0034614e-05, 2.9122733e-04],
       [1.5092069e-09, 6.2632665e-26, 9.9999821e-01, ..., 1.2528053e-16, 5.1928947e-15, 2.4766043e-09],
       ...,
       [2.3087073e-22, 8.9350849e-30, 1.5581450e-06, ..., 4.6803438e-30, 2.6182833e-38, 9.9999845e-01],
       [1.2802441e-08, 4.8800103e-23, 9.9999595e-01, ..., 1.9499300e-14, 4.1101389e-13, 1.3439305e-08],
       [1.9302787e-03, 1.1111683e-06, 9.5764957e-03, ..., 3.9356881e-05, 8.9573540e-04, 6.0943735e-04]], dtype=float32)